<a href="https://colab.research.google.com/github/jyotidabass/Mistral_7B-chatbot_app/blob/main/Mistral_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mistral 7B

Mistral 7B is a new state-of-the-art open-source model. Here are some interesting facts about it

* One of the strongest open-source models, of all sizes
* Strongest model in the 1-20B parameter range models
* Does decently in code-related tasks
* Uses Windowed attention, allowing to push to 200k tokens of context if using Rope (needs 4 A10G GPUs for this)
* Apache 2.0 license

As for the integrations status:
* Integrated into `transformers`
* You can use it with a server or locally (it's a small model after all!)
* Integrated into popular tools tuch as TGI and VLLM


Two models are released: a [base model](https://huggingface.co/mistralai/Mistral-7B-v0.1) and a [instruct fine-tuned version](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1). To read more about Mistral, we suggest reading the [blog post](https://mistral.ai/news/announcing-mistral-7b/).

In this Colab, we'll experiment with the Mistral model using an API. There are three ways we can use it:

* **Free API:** Hugging Face provides a free Inference API for all its users to try out models. This API is rate limited but is great for quick experiments.
* **PRO and Enterprise API:** Hugging Face provides an open API for all its PRO users.  Subscribing to the Pro Inference API costs $9/month and allows you to experiment with many large models, such as Llama 2 and SDXL. Read more about it [here](https://huggingface.co/blog/inference-pro).
* **Inference Endpoints:** For enterprise and production-ready cases. You can deploy it with 1 click [here](https://ui.endpoints.huggingface.co/catalog).

This demo does not require GPU Colab, just CPU. You can grab your token at https://huggingface.co/settings/tokens.

**This colab shows how to use HTTP requests as well as building your own chat demo for Mistral.**

## Doing curl requests


In this notebook, we'll experiment with the instruct model, as it is trained for instructions. As per [the model card](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1), the expected format for a prompt is as follows

From the model card

> In order to leverage instruction fine-tuning, your prompt should be surrounded by [INST] and [\INST] tokens. The very first instruction should begin with a begin of sentence id. The next instructions should not. The assistant generation will be ended by the end-of-sentence token id.

```
<s>[INST] {{ user_msg_1 }} [/INST] {{ model_answer_1 }}</s> [INST] {{ user_msg_2 }} [/INST] {{ model_answer_2 }}</s>
```

Note that models can be quite reactive to different prompt structure than the one used for training, so watch out for spaces and other things!

We'll start an initial query without prompt formatting, which works ok for simple queries.

In [1]:
!curl https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1 \
  --header "Content-Type: application/json" \
	-X POST \
	-d '{"inputs": "Explain ML as a pirate", "parameters": {"max_new_tokens": 50}}' \
	-H "Authorization: Bearer API_TOKEN"

{"error":"Authorization header is correct, but the token seems invalid"}

## Programmatic usage with Python

You can do simple `requests`, but the `huggingface_hub` library provides nice utilities to easily use the model. Among the things we can use are:

* `InferenceClient` and `AsyncInferenceClient` to perform inference either in a sync or async way.
* Token streaming: Only load the tokens that are needed
* Easily configure generation params, such as `temperature`, nucleus sampling (`top-p`), repetition penalty, stop sequences, and more.
* Obtain details of the generation (such as the probability of each token or whether a token is the last token).

In [2]:
%%capture
!pip install huggingface_hub gradio

In [3]:
from huggingface_hub import InferenceClient

client = InferenceClient(
    "mistralai/Mistral-7B-Instruct-v0.1"
)

prompt = """<s>[INST] What is your favourite condiment?  [/INST]</s>
"""

res = client.text_generation(prompt, max_new_tokens=95)
print(res)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



I don't have personal experiences or preferences. However, I can tell you that people's favorite condiments can vary widely based on personal taste and cultural background. Some popular condiments include ketchup, mustard, mayonnaise, hot sauce, soy sauce, and olive oil.


We can also use [token streaming](https://huggingface.co/docs/text-generation-inference/conceptual/streaming). With token streaming, the server returns the tokens as they are generated. Just add `stream=True`.

In [4]:
res = client.text_generation(prompt, max_new_tokens=35, stream=True, details=True, return_full_text=False)
for r in res: # this is a generator
  # print the token for example
  print(r)
  continue

TextGenerationStreamResponse(token=Token(id=13, text='\n', logprob=-0.45092773, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=28737, text='I', logprob=-1.5058594, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=949, text=' don', logprob=-1.3085938, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=28742, text="'", logprob=-0.01713562, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=28707, text='t', logprob=-1.2755394e-05, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=506, text=' have', logprob=-0.5395508, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=3327, text=' personal', logprob=-0.33081055, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=9021, text=' expe

Let's now try a multi-prompt structure

In [5]:
def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

In [6]:
message = "And what do you think about it?"
history = [["What is your favourite condiment?", "My favorite condiment is ketchup. It's versatile, tasty, and goes well with a variety of foods."]]

format_prompt(message, history)

"<s>[INST] What is your favourite condiment? [/INST] My favorite condiment is ketchup. It's versatile, tasty, and goes well with a variety of foods.</s> [INST] And what do you think about it? [/INST]"

## End-to-end demo

Let's now build a Gradio demo that takes care of:

* Handling multiple turns of conversation
* Format the prompt in correct structure
* Allow user to specify/modify the parameters
* Stop the generation

Just run the following cell and have fun!

In [7]:
!pip install gradio

In [8]:
import gradio as gr

def generate(
    prompt, history, temperature=0.9, max_new_tokens=256, top_p=0.95, repetition_penalty=1.0,
):
    temperature = float(temperature)
    if temperature < 1e-2:
        temperature = 1e-2
    top_p = float(top_p)

    generate_kwargs = dict(
        temperature=temperature,
        max_new_tokens=max_new_tokens,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=True,
        seed=42,
    )

    formatted_prompt = format_prompt(prompt, history)

    stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=True, details=True, return_full_text=False)
    output = ""

    for response in stream:
        output += response.token.text
        yield output
    return output


additional_inputs=[
    gr.Slider(
        label="Temperature",
        value=0.9,
        minimum=0.0,
        maximum=1.0,
        step=0.05,
        interactive=True,
        info="Higher values produce more diverse outputs",
    ),
    gr.Slider(
        label="Max new tokens",
        value=256,
        minimum=0,
        maximum=8192,
        step=64,
        interactive=True,
        info="The maximum numbers of new tokens",
    ),
    gr.Slider(
        label="Top-p (nucleus sampling)",
        value=0.90,
        minimum=0.0,
        maximum=1,
        step=0.05,
        interactive=True,
        info="Higher values sample more low-probability tokens",
    ),
    gr.Slider(
        label="Repetition penalty",
        value=1.2,
        minimum=1.0,
        maximum=2.0,
        step=0.05,
        interactive=True,
        info="Penalize repeated tokens",
    )
]

with gr.Blocks() as demo:
    gr.ChatInterface(
        generate,
        additional_inputs=additional_inputs,
    )

demo.queue().launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://908fd5c7378e19d03f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://908fd5c7378e19d03f.gradio.live


## What's next?

* Try out Mistral 7B in this [free online Space](https://huggingface.co/spaces/osanseviero/mistral-super-fast)
* Deploy Mistral 7B Instruct with one click [here](https://ui.endpoints.huggingface.co/catalog)
* Deploy in your own hardware using https://github.com/huggingface/text-generation-inference
* Run the model locally using `transformers`